Configuración inicial

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

spark = SparkSession.builder \
  .master('local') \
  .appName('Lemon Data Challenge') \
  .config('spark.some.config.option', 'some.value') \
  .getOrCreate()

In [3]:
rates = spark.read.csv('rates.csv', header = True)
transactions = spark.read.csv('transactions.csv', header = True)
users = spark.read.csv('users.csv', header = True)

In [4]:
rates.show()
transactions.show()
users.show()

+-------------+--------------+--------------+
|base_currency|quote_currency|         price|
+-------------+--------------+--------------+
|          UST|          USDT|    1.00000000|
|         SAND|           BTC|    0.00006944|
|         USDC|          USDT|    0.99980000|
|          UNI|           BTC|    0.00023050|
|          ADA|           ETH|    0.00032330|
|         LUNA|           ETH|    0.02965000|
|          SOL|           ETH|    0.03499000|
|          SOL|           BTC|    0.00265530|
|          SOL|          USDT|  113.31000000|
|          ETH|           BTC|    0.07587900|
|          BTC|          USDC|42663.01000000|
|          SLP|          USDT|    0.01980000|
|          DOT|           ETH|    0.00612100|
|          ETH|           DAI| 3237.70000000|
|          ADA|           BTC|    0.00002453|
|          SOL|          USDC|  113.31000000|
|         SAND|          USDT|    2.96240000|
|          AXS|           ETH|    0.01598000|
|          BTC|           DAI|4266

# **CHURN**

Transacciones Febrero

In [5]:
transactions.createTempView('transacciones_feb')

In [8]:
dates = "DATE(createdat) >= '2022-02-01' AND DATE(createdat) < '2022-03-01'"
transacciones_febrero = spark.sql(f'SELECT DISTINCT(user_id) AS user_id FROM transacciones_feb WHERE {dates}')

Transacciones Marzo

In [9]:
transactions.createTempView('transacciones_mar')

In [10]:
dates = "DATE(createdat) >= '2022-03-01' AND DATE(createdat) < '2022-04-01'"
transacciones_marzo = spark.sql(f'SELECT DISTINCT(user_id) AS user_id FROM transacciones_mar WHERE {dates}')

Usuarios que realizaron transacción en Febrero pero no en Marzo

In [11]:
transacciones_febrero.createTempView('trans_feb')
transacciones_marzo.createTempView('trans_mar')

In [12]:
usuarios_churn = spark.sql("SELECT user_id FROM trans_feb WHERE user_id NOT IN (SELECT user_id FROM trans_mar)").count()
usuarios_churn

48

CHURN

In [13]:
churn = usuarios_churn / transacciones_marzo.count()
churn

0.1875

In [14]:
retencion = 1 - churn
retencion

0.8125

# **ARPU**

In [15]:
transactions.createTempView('trans')
rates.createTempView('rates')

Tomo el precio en USDT de cada moneda y lo uno a la tabla de transacciones

In [38]:
col = "user_id, amount, transaction_type, currency, price"
trans = spark.sql(("""SELECT {} FROM trans AS t 
                  LEFT JOIN(SELECT base_currency, price FROM rates
                  WHERE quote_currency = 'USDT') AS r ON t.currency = r.base_currency"""
                  .format(col)))
trans.show(5)

+-------+-------------+------------------+--------+-----+
|user_id|       amount|  transaction_type|currency|price|
+-------+-------------+------------------+--------+-----+
|3222205| 100.02000000|       CRYPTO_SALE|   MONEY| null|
|1058316|1060.00000000|LEMON_CARD_PAYMENT|   MONEY| null|
|3245984| 349.27000000|       CRYPTO_SALE|   MONEY| null|
|4430720|1440.00000000|LEMON_CARD_PAYMENT|   MONEY| null|
|3193610| 100.00000000|   CRYPTO_PURCHASE|   MONEY| null|
+-------+-------------+------------------+--------+-----+
only showing top 5 rows



In [39]:
trans = trans.withColumn('amount_usd',
        f.when((f.col('currency') == 'MONEY')|(f.col('currency') == 'DAI')|
               (f.col('currency') == 'USDT')|
               (f.col('currency') == 'UST'),
               f.col('amount')).otherwise(trans.amount * trans.price)
        )
trans.show(5)

+-------+-------------+------------------+--------+-----+-------------+
|user_id|       amount|  transaction_type|currency|price|   amount_usd|
+-------+-------------+------------------+--------+-----+-------------+
|3222205| 100.02000000|       CRYPTO_SALE|   MONEY| null| 100.02000000|
|1058316|1060.00000000|LEMON_CARD_PAYMENT|   MONEY| null|1060.00000000|
|3245984| 349.27000000|       CRYPTO_SALE|   MONEY| null| 349.27000000|
|4430720|1440.00000000|LEMON_CARD_PAYMENT|   MONEY| null|1440.00000000|
|3193610| 100.00000000|   CRYPTO_PURCHASE|   MONEY| null| 100.00000000|
+-------+-------------+------------------+--------+-----+-------------+
only showing top 5 rows



Identifico los distintos tipos de transacciones para luego calcular el revenue

In [31]:
transactions_type = spark.sql("SELECT DISTINCT(transaction_type) FROM trans")
transactions_type.show()

+------------------+
|  transaction_type|
+------------------+
|LEMON_CARD_PAYMENT|
|VIRTUAL_WITHDRAWAL|
|   VIRTUAL_DEPOSIT|
|    CASH_IN_CRYPTO|
|       CRYPTO_SALE|
|       CRYPTO_SWAP|
|   CRYPTO_PURCHASE|
+------------------+



In [32]:
trans = trans.withColumn('revenue_amount',
        f.when(f.col('transaction_type') == 'CRYPTO_SWAP', trans.amount_usd * 0.01)
          .otherwise(f.when(f.col('transaction_type') == 'CRYPTO_SALE', trans.amount_usd * 0.02)
            .otherwise(f.when(f.col('transaction_type') == 'CRYPTO_PURCHASE', trans.amount_usd * 0.02)
              .otherwise(f.when(f.col('transaction_type') == 'LEMON_CARD_PAYMENT', trans.amount_usd * 0.05)
                .otherwise(0)))))
trans.show(5)

+-------+-------------+------------------+--------+-----+-------------+-----------------+
|user_id|       amount|  transaction_type|currency|price|   amount_usd|   revenue_amount|
+-------+-------------+------------------+--------+-----+-------------+-----------------+
|3222205| 100.02000000|       CRYPTO_SALE|   MONEY| null| 100.02000000|           2.0004|
|1058316|1060.00000000|LEMON_CARD_PAYMENT|   MONEY| null|1060.00000000|             53.0|
|3245984| 349.27000000|       CRYPTO_SALE|   MONEY| null| 349.27000000|6.985399999999999|
|4430720|1440.00000000|LEMON_CARD_PAYMENT|   MONEY| null|1440.00000000|             72.0|
|3193610| 100.00000000|   CRYPTO_PURCHASE|   MONEY| null| 100.00000000|              2.0|
+-------+-------------+------------------+--------+-----+-------------+-----------------+
only showing top 5 rows



Revenue y Usuarios totales

In [33]:
trans.createTempView('revenue_total')
users.createTempView('usuarios_totales')

In [34]:
revenue_total = spark.sql("SELECT SUM(revenue_amount) AS revenue_total FROM revenue_total")
revenue_total.show()

+-----------------+
|    revenue_total|
+-----------------+
|461248.0122836366|
+-----------------+



In [35]:
usuarios_totales = spark.sql("SELECT COUNT(DISTINCT(user_id)) AS usuarios_totales FROM usuarios_totales")
usuarios_totales.show()

+----------------+
|usuarios_totales|
+----------------+
|           50000|
+----------------+



ARPU

In [36]:
r_tot = revenue_total.collect()[0][0]
u_tot = usuarios_totales.collect()[0][0]

arpu = r_tot / u_tot
arpu

9.224960245672731

# **LTV**

In [37]:
LTV = arpu / churn
LTV

49.199787976921236